In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import scipy
import math

In [ ]:
print(f"networkx = {nx.__version__}")
print(f"numpy    = {np.__version__}")
print(f"pandas   = {pd.__version__}")
print(f"scipy    = {scipy.__version__}")

# Structure of Networks 

- Which networks form?
    - random graph models ‐ "How?"
    - Economic/game theoretic models ‐ "Why?"
- How does it depend on context?

## Static Random networks

- Useful Benchmark
    - component structure
    - diameter
    - degree distribution
    - clustering...

- Tools and methods
    - properties and thresholds

- Properties of Networks
    - Every network has some probability of forming
    - How to make sense of that?
    - Examine what happens for "large" networks

- Specifying Properties
    - $G(N) =$ all the undirected networks on the set of nodes N
    - A property is a set $A(N)$ for each $N$ such that $A(N)$ is a subset of $G(N)$
        - a specification of which networks have that property

- Examples of Properties
    - $A(N)=\{g | N_i (g)$ nonempty for all $i \in N\}$
        - property of no isolated nodes
    - $A(N)=\{g | l (i,j)$ finite for all $i,j \in N\}$
        - network is connected
    - $A(N)=\{g | l (i,j) < \log(n)$ for all $i,j \in N\}$
        - diameter is less than $\log (n)$

- Monotone Properties
    - A property $A(N)$ is monotone if $g \in A(N)$ and $g \subset g'$ implies $g' \in A(N)$.
    - All three of the previous properties are monotone

- Limiting Properties
    - In order to deduce things about random networks, we often look at "large" networks, by examining limits
    - Deduce things about properties as $n \to \infty$

- Threshold Functions and Phase Transitions
    - t(n) is a threshold function for a monotone property
$A(N)$ if
$P[A(N) | p(n) ] \to 1$ if $p(n)/t(n) \to \infty$
and
$P[A(N) | p(n) ] \to 0$ if $p(n)/t(n) \to 0$
    - A phase transition occurs at $t(n)$

In [ ]:
def average_degree(graph):
    return np.mean([x[1] for x in graph.degree()])

def overall_clustering(graph):
    triangles = nx.triangles(graph)
    degree = nx.degree(graph)
    try:
        return sum([triangles[k] for k in triangles]) / sum([v * (v - 1) / 2 for k, v in degree])
    except:
        return 0

def graph_stat(graph):
    largest_comp = graph.subgraph(max(nx.connected_components(graph), key=lambda x: len(x)))
    return {
        "number_of_nodes": graph.number_of_nodes(),
        "number_connected_components": nx.number_connected_components(graph),
        "number_of_nodes_in_largest_component": largest_comp.number_of_nodes(),
        "diameter": nx.diameter(largest_comp),
        "numb_cycles": len(nx.cycle_basis(graph)),
        "average_shortest_path_length": nx.average_shortest_path_length(largest_comp),
        "average_degree": average_degree(graph),
        "average_clustering": nx.average_clustering(graph),
        "overall_clustering": overall_clustering(graph),
    }

### Erdos‐Renyi Random Graphs

- start with $n$ nodes
- each link is formed independently with some probability $p$
- Serves as a benchmark $G(n,p)$

In [ ]:
graph_random = nx.gnp_random_graph(50, 0.05)
nx.draw(graph_random)

In [ ]:
graph_stat(graph_random)

#### Summary statistics dynamics

In [ ]:
def bootstraped_stat_random(n, p, instances=100):
    return pd.DataFrame([graph_stat(nx.gnp_random_graph(n, p)) for _ in range(instances)]).median(axis=0)

In [ ]:
bootstraped_stat_random(50, 0.0001) # no connection between nodes

In [ ]:
bootstraped_stat_random(50, 0.001) # the network has some links

In [ ]:
bootstraped_stat_random(50, 0.01) # the network has a component with at least three links

In [ ]:
bootstraped_stat_random(50, 0.05) # the network has a cycle

In [ ]:
bootstraped_stat_random(50, 0.1) # the network is connected

#### Thresholds for Erdos‐Renyi Random Graphs

- $1/n^2$ ‐ the network has some links (avg deg $1/n$)
- $1/n^{\frac{3}{2}}$ – the network has a component with at least
three links (avg deg $1/n^{\frac{1}{2}}$ )
- $1/n$ – the network has a cycle, the network has a unique
giant component: a component with at least $n^a$ nodes for
some fixed $a<1$; (avg deg 1)
- $\log(n)/n$ ‐ the network is connected; (avg deg $\log(n)$)

### Erdos‐Renyi Random Graphs drawbacks

#### Degree distribution

- probability that node has d links is binomial
$$\frac{(n‐1)!}{(d!(n‐d‐1)!)} p^d (1‐p)^{n‐d‐1}$$
- Large n, small p, this is approximately a Poisson distribution:
$$\frac{(n‐1)^d}{d!} p^d e^{‐(n‐1)p}$$
- hence name ``Poisson random graphs’’

In [ ]:

def plot_random_graph_degree_distribution(n, p):
    values, counts = np.unique([x[1] for x in nx.gnp_random_graph(n, p).degree()], return_counts=True)
    plt.plot(values, counts / np.sum(counts), label = "Realized frequency")

    values, counts = np.unique(scipy.stats.poisson.rvs(mu=(n-1)*p, size=5000), return_counts=True)
    plt.plot(values, counts / np.sum(counts), label = "Poisson approximation")
    plt.legend()

    plt.show()

In [ ]:
plot_random_graph_degree_distribution(200, 0.03)

Many real-world examples with fat tails distribution of links per node:  

- Price DJS. 1965. Networks of scientific papers. Science 149: 510 15
    - More high and low degree nodes than predicted at random
    - Citation Networks ‐ too many with 0 citations,
    too many with high numbers of citations to have
    citations drawn at random
    - "Fat tails" compared to random network
- Pareto, V. (1896) Cours d’Economie Politique, Geneva: Droz
- Yule, G. (1925) “A Mathematical Theory of Evolution Based on the Conclusions of Dr. J.C. Willis,” F.R.S. Philosophical Transactions of the Royal Society of London B 213:21–87.
- Zipf, G. (1949) Human Behavior and the Principle of Least Effort, Cambridge, Mass.: Addison‐Wesley.
- Simon, H. (1955) “On a Class of Skew Distribution Functions,” Biometrika 42(3–4):425–440.

#### Clustering

- Average and Overall clustering tend to 0, if max degree is bounded and network becomes large:
$$Cl(g) =\frac{\sum_i \#\{ kj \in g | k, j \in N_i(g)\}}{\sum_i \#\{ kj | k, j \in N_i(g)\}}=p$$
- If degree is bounded, then $p(n‐1)$ is bounded
- So $p$ goes to 0 as $n$ grows

Real world examples - clustering Coefficients vs probability for link

- Prison friendships
    - .31 vs .0134
- co‐authorships
    - .15 math vs .00002,
    - .09 biology vs .00001,
    - .19 econ vs .00002,
- www
    - .11 for web links vs .0002

### Few Examples

In [ ]:
def degree_df(graph):
    values, counts = np.unique(sorted([x[1] for x in graph.degree()], reverse=True), return_counts=True)
    total_counts = np.sum(counts)
    freq = counts / total_counts
    cum_freq = np.cumsum(counts) / total_counts
    return {
        "degree": values,
        "freq": freq,
        "cum_freq": cum_freq
    }

def plot_ddf_comparison(graph1, graph2, graph3=None):
    ddf1 = degree_df(graph1)
    ddf2 = degree_df(graph2)
    plt.plot(np.log(ddf1["degree"][0:-1]), np.log(1 - ddf1["cum_freq"][0:-1]), label="graph 1")
    plt.plot(np.log(ddf2["degree"][0:-1]), np.log(1 - ddf2["cum_freq"][0:-1]), label="graph 2")
    if graph3:
        ddf3 = degree_df(graph3)
        plt.plot(np.log(ddf3["degree"][0:-1]), np.log(1 - ddf3["cum_freq"][0:-1]), label="graph 3")

    plt.xlabel("log(degree)")
    plt.ylabel("log(cum distribution)")
    plt.legend()
    plt.show()

#### Zachary's karate club
A social network of a karate club was studied by Wayne W. Zachary for a period of three years from 1970 to 1972. The network captures 34 members of a karate club, documenting links between pairs of members who interacted outside the club. During the study a conflict arose between the administrator "John A" and instructor "Mr. Hi" (pseudonyms), which led to the split of the club into two. Half of the members formed a new club around Mr. Hi; members from the other part found a new instructor or gave up karate. Based on collected data Zachary correctly assigned all but one member of the club to the groups they actually joined after the split.

In [ ]:
graph_kc = nx.karate_club_graph()
graph_stat(graph_kc)

In [ ]:
bootstraped_stat_random(34, 4.58 / (34 - 1))

In [ ]:
graph_kc_random_sim = nx.gnp_random_graph(34, 4.58 / (34 - 1))
graph_stat(graph_kc_random_sim)

In [ ]:
plot_ddf_comparison(graph_kc, graph_kc_random_sim)

#### Florentine Families

The marriage links of Florentine families

In [ ]:
graph_ff = nx.florentine_families_graph()
graph_stat(graph_ff)

In [ ]:
bootstraped_stat_random(15, 2.66 / (15-1))

In [ ]:
graph_ff_random_sim = nx.gnp_random_graph(15, 2.66 / (15-1))
graph_stat(graph_ff_random_sim)

In [ ]:
plot_ddf_comparison(graph_ff, graph_ff_random_sim)

#### Davis Southern women social network
The Davis Southern women social network is a dataset collected by Davis and a colleague in the 1930s. It contains the observed attendance at 14 social events by 18 Southern women. A vertex annotation "Identity" describes the type of vertex.

In [ ]:
graph_dsw = nx.davis_southern_women_graph()
graph_stat(graph_dsw)

In [ ]:
graph_dsw_random_sim = nx.gnp_random_graph(32, 5.56 / (32-1))
graph_stat(graph_dsw_random_sim)

In [ ]:
plot_ddf_comparison(graph_dsw, graph_dsw_random_sim)

### Rewired lattice ‐ Watts and Strogatz 98

- Erdos‐Renyi model misses clustering
    - clustering is on the order of p; going to 0 unless average degree is becoming infinite (and highly so...)
- Start with ring‐lattice and then randomly pick some links to rewire
    - start with high clustering but high diameter
    - as rewire enough links, get low diameter
    - don’t rewire too many, keep high clustering

In [ ]:
graph_ws = nx.watts_strogatz_graph(50, 4, 0.05)
nx.draw_circular(graph_ws)



In [ ]:
nx.draw(graph_ws)

In [ ]:
def bootstraped_stat_ws(n, k, p, instances=100):
    return pd.DataFrame([graph_stat(nx.watts_strogatz_graph(n, k, p)) for _ in range(instances)]).median(axis=0)

In [ ]:
bootstraped_stat_ws(50, 4, 0.001)

In [ ]:
bootstraped_stat_ws(50, 4, 0.01)

In [ ]:
bootstraped_stat_ws(50, 4, 0.1)

In [ ]:
bootstraped_stat_ws(50, 4, 0.25)

### Newman, Watts, Strogatz 99

In [ ]:
def bootstraped_stat_nws(n, k, p, instances=100):
    return pd.DataFrame([graph_stat(nx.newman_watts_strogatz_graph(n, k, p)) for _ in range(instances)]).median(axis=0)

In [ ]:
bootstraped_stat_nws(50, 4, 0.001)

In [ ]:
bootstraped_stat_nws(50, 4, 0.01)

In [ ]:
bootstraped_stat_nws(50, 4, 0.1)

In [ ]:
bootstraped_stat_nws(50, 4, 0.25)

### Few Examples

#### Zachary's karate club

In [ ]:
graph_stat(graph_kc)

In [ ]:
bootstraped_stat_nws(34, 4, (4.58 - 4) / 4)

In [ ]:
graph_kc_nws_sim = nx.newman_watts_strogatz_graph(34, 4, (4.58 - 4) / 4)
graph_stat(graph_kc_nws_sim)

In [ ]:
plot_ddf_comparison(graph_kc, graph_kc_nws_sim)

### Growing Random Networks

- Motivation
    - Citation networks
    - Web
    - Scientific networks
    - Societies...

- What do they add?
    - Realism(?)
    - Natural form of heterogeneity via age
    - A form of dynamics
    - Natural way of varying degree distributions
        - not pre‐specified as in static models

#### Growing and Uniformly Random
- Each date a new node is born
- Forms m links to existing nodes
- Each node is chosen with equal likelihood

In [ ]:
def grn_uniform(n, m):
    graph = nx.complete_graph(m)
    for i in range(m, n):
        new_edges = [(x, i) for x in random.sample(list(graph.nodes), m)]
        graph.add_edges_from(new_edges)
    return graph
        

In [ ]:
gnr = grn_uniform(50, 2)
nx.draw(gnr)

In [ ]:
pd.DataFrame([graph_stat(grn_uniform(50, 4)) for _ in range(100)]).median(axis=0)

#### Growing and Uniformly Random - Degree distribution

In [ ]:
pd.DataFrame([dict(grn_uniform(120, 20).degree()) for _ in range(1000)]).median(axis=0).plot(xlabel="Born time of the node", ylabel="Degree")

In [ ]:
pd.DataFrame([dict(grn_uniform(220, 20).degree()) for _ in range(1000)]).median(axis=0).plot(xlabel="Born time of the node", ylabel="Degree")

- Expected degree for node $i$ born at $m<i<t$ is $$d_i(t)=m + m/(i+1) + m/(i+2) + ... + m/ t$$ or $d_i(t)=m(1+\log(t/i))$ (Harmonic numbers)
- Nodes that have expected degree less than $d$ at some time $t$ are those such that $m(1+\log(t/i)) < d$, i.e.
$$i > t e^{-\frac{d-m}{m}}$$
- Hence, degree distribution is
$$F_t(d) =(t - t e^{-\frac{d-m}{m}})/t = 1- e^{-\frac{d-m}{m}}$$
- Distribution of expected degrees is such that $d-m$ is exponentially distributed (mean $m$)
- Good approximation for large $t$
- Fat tails are not completly addressed

#### Few examples

##### Zachary's karate club

In [ ]:
graph_stat(graph_kc)

In [ ]:
graph_kc_grn_uni_sim = grn_uniform(34, math.ceil(4.5882 / 2))
graph_stat(graph_kc_grn_uni_sim)

In [ ]:
plot_ddf_comparison(graph_kc, graph_kc_grn_uni_sim)

##### Florentine Families

In [ ]:
graph_stat(graph_ff)

In [ ]:
graph_ff_grn_uni_sim = grn_uniform(15, math.ceil(2.666 / 2))
graph_stat(graph_ff_grn_uni_sim)

In [ ]:
plot_ddf_comparison(graph_ff, graph_ff_grn_uni_sim)

#### Preferential Attachment

- Newborn nodes form $m$ links to existing nodes
- $tm$ links in total
- total degree is $2tm$
- $d_i(t)$ degree of node born at time $i$ at time $t$
- Probability of attaching to $i$ is $d_i(t)/2tm$

In [ ]:
gnr_ba = nx.barabasi_albert_graph(50, 2)
nx.draw(gnr_ba)

In [ ]:
pd.DataFrame([graph_stat(nx.barabasi_albert_graph(50, 4)) for _ in range(100)]).median(axis=0)

In [ ]:
plot_ddf_comparison(gnr, gnr_ba)

#### Few examples

##### Zachary's karate club

In [ ]:
graph_stat(graph_kc)

In [ ]:
graph_kc_grn_ba_sim = nx.barabasi_albert_graph(34, 3)
graph_stat(graph_kc_grn_ba_sim)

In [ ]:
plot_ddf_comparison(graph_kc, graph_kc_grn_ba_sim, graph_kc_grn_uni_sim)

##### Florentine Families

In [ ]:
graph_stat(graph_ff)

In [ ]:
graph_ff_grn_ba_sim = nx.barabasi_albert_graph(15, 2)
graph_stat(graph_ff_grn_ba_sim)

In [ ]:
plot_ddf_comparison(graph_ff, graph_ff_grn_ba_sim, graph_ff_grn_uni_sim)

#### Preferential Attachment - Degree distribution

- new links gained per unit time
$$\frac{dd_i(t)}{dt} = m(d_i(t)/2tm) = d_i(t)/2t$$
- starting condition
$$d_i(i)=m$$
- Solution of the differential equation is
$$d_i(t) = m (t/i)^{\frac{1}{2}}$$
- Nodes that have expected degree less than $d$ at
some time $t$ are those such that $m (t/i)^{\frac{1}{2}} < d$, i.e.
$$i > t \frac{m^2}{d^2}$$
$$F_t(d) =\left(t- t \frac{m^2}{d^2} \right)/t = 1-\frac{m^2}{d^2}$$

#### Hybrid model 

- Fraction $\alpha$ uniformly at random, $1‐\alpha$ via preferential
attachment
- new links gained per unit time
$$\frac{dd_i(t)}{dt} = \alpha m/t + (1-\alpha)d_i(t)/2t$$
- Starting condition $$d_i(i)=m$$
- Solution of the differential equation is
$$d_i(t) = (m + 2\alpha m/(1-\alpha))(t/i)^\frac{1-\alpha}{2} - 2\alpha m/(1-\alpha)$$
- Nodes that have expected degree less than $d$ at some time $t$ are those $i$ such that
$$(m + x\alpha m)(t/i)^{\frac{1}{x}} - x\alpha m < d \text{ where } x = \frac{2}{1-\alpha}$$
- critical $i$ is such that
$$\frac{i}{t} = [(m + x\alpha m)/ (d + x\alpha m)]^x$$
- Degree Distribution
$$F(d) = \frac{t – i}{t} = 1 – ((m+\alpha mx)/(d+\alpha mx))^x\text{ where } x = \frac{2}{1-\alpha}$$
- Spans Extremes
    - $\alpha$ near 1 nearly exponential
    - $\alpha$ near 0 nearly preferential

In [ ]:
def hybrid_ddf(alpha, d, m):
    if alpha == 1:
        return np.maximum(1- np.exp(-(d-m)/m), 0)
    x = 2 / (1 - alpha)
    amx = alpha * m * x
    return np.maximum(1 - (((m + amx) / (d + amx)) ** x), 0)

def fit_hybrid_model(graph, m):
    ddf = degree_df(graph)

    def loss_fun(alpha):
        ddf_est = hybrid_ddf(alpha, ddf["degree"], m)
        return np.sum((ddf["cum_freq"] - ddf_est) ** 2)

    return scipy.optimize.minimize(loss_fun, 0.5, bounds=scipy.optimize.Bounds(lb=0, ub=0.99))

    # return loss_fun(0.5)

In [ ]:
m = 2
d = np.array(range(m+1,50))
ddf_1 = hybrid_ddf(1, d, m)
ddf_0 = hybrid_ddf(0, d, m)
ddf_05 = hybrid_ddf(0.5, d, m)

In [ ]:
plt.plot(np.log(d), np.log(1 - ddf_1), label="alpha = 1")
plt.plot(np.log(d), np.log(1 - ddf_05), label="alpha = 0.5")
plt.plot(np.log(d), np.log(1 - ddf_0), label="alpha = 0")
plt.legend()

plt.show()

In [ ]:
fit_hybrid_model(graph_kc, 3)

In [ ]:
fit_hybrid_model(graph_ff, 2)

#### Stochastic block models

- Extend the basic Erdos‐Renyi G(n,p) model
- Nodes have characteristics e.g., age, gender, religion, profession, etc.
- links between nodes depend on the pairs’ characteristics
- Example: link between $i$ and $j$ depends on their characteristics:
$$\log\left( \frac{p_{ij}}{1-p_{ij}}\right) = \beta_i X_i + \beta_j X_j + β_{ij} |X_i ‐ X_j|$$
- Could use this sort of model to test for homophily...

In [ ]:
# graph used for illustrations
edges_list = [
    (1, 2), (1, 3), (2, 3), #yellow group
    (4, 5), (4, 6), (4, 7), (5, 6), (6, 7), #blue group
    (2, 4), (3, 5), # yellow to blue
    (8, 9), (8, 10), (9, 10), (9, 11), (10, 11), #yellow group
    (4, 8), (7, 8),
    (3, 11)
]
graph_block = nx.from_edgelist(edges_list)

pos = nx.spring_layout(graph_block)
# draw nodes
nx.draw_networkx_nodes(graph_block.subgraph([1, 2, 3]), pos, node_color="yellow")
nx.draw_networkx_nodes(graph_block.subgraph([4, 5, 6, 7]), pos, node_color="blue")
nx.draw_networkx_nodes(graph_block.subgraph([8, 9, 10, 11]), pos, node_color="green")
# draw nodes'label
# nx.draw_networkx_labels(graph_block, pos, font_size=10)

nx.draw_networkx_edges(graph_block, pos)
# nx.draw(graph_centrality, with_labels=True)

Probabilities
- blue - blue -> 5/6
- yellow - yellow - 3/3
- green - green - 5/6
- blue - yellow - 2/12
- blue - green - 2/16
- yellow - green - 1/12

In [ ]:
sizes = [3, 4, 4]
probs = [[1, 1/6, 1/12], [1/6, 5/6, 1/8], [1/12, 1/8, 5/6]]
g = nx.stochastic_block_model(sizes, probs)

In [ ]:
# define nodes' position
# pos = nx.layout(g)
pos = nx.spring_layout(g)
# draw nodes
nx.draw_networkx_nodes(g.subgraph(g.graph["partition"][0]), pos, node_color="yellow")
nx.draw_networkx_nodes(g.subgraph(g.graph["partition"][1]), pos, node_color="blue")
nx.draw_networkx_nodes(g.subgraph(g.graph["partition"][2]), pos, node_color="green")
# draw nodes'label
nx.draw_networkx_labels(g, pos, font_size=10)
# draw edges as arcs, see connection style documentation for more options
nx.draw_networkx_edges(g, pos, edgelist=nx.to_edgelist(g), width=1) #,connectionstyle="arc3,rad=-0.2")

# Homework

Table 1

|Percent:|52    |38    |5        |     5|
|--------|------|------|---------|------|
|        |White |Black |Hispanic |Other |
|White| 86| 7| 47| 74|
|Black| 4 | 85| 46| 13|
|Hispanic| 4| 6| 2| 4|
|Other| 6| 2| 5| 9|
||100 |100 |100| 100|

Table 2

| |n=850|n=62|n=75|n=100|n=230|
|-|-----|----|----|-----|-----|
||Dutch| Moroccan| Turkish| Surinamese| Other|
|Dutch| 79| 24| 11| 21| 47|
|Moroccan| 2| 27| 8| 4| 5|
|Turkish |2 |19 |59 |8 |6|
|Surinamese| 3| 8| 8| 44| 12|
|Other| 13| 22| 14| 23| 30|
| |100| 100| 100| 100| 100|

Write a python function which to find the probalities to have a link between the different groups in the tables above. Simulate block models which to represent them.

In [ ]:
color = {
    1: "yellow",
    2: "yellow",
    3: "yellow",
    4: "blue",
    5: "blue",
    6: "blue",
    7: "blue",
    8: "green",
    9: "green",
    10: "green",
    11: "green"
}
nx.set_node_attributes(graph_block, color, "color")

In [ ]:
graph_block.nodes.data()

In [ ]:
import itertools

In [ ]:
def create_connection_data(graph):
    graph_data = graph.nodes.data()
    all_prop = {y for x in graph.nodes.data() for y in x[1].keys()}
    all_comb = [
        {
            # "index": comb,
            "index": (x[0], y[0]),
            "connected": int((x[0], y[0]) in graph.edges()),
            **{
                f"{prop}_left": graph_data[x[0]][prop]
                for prop in all_prop
            },
            **{
                f"{prop}_right": graph_data[y[0]][prop]
                for prop in all_prop
            }
        }
        # for comb in itertools.combinations(graph.nodes, 2)
        for x in graph.nodes.data()
        for y in graph.nodes.data() if x != y
    ]
    result = pd.DataFrame(all_comb)
    for prop in all_prop:
        if result[f"{prop}_left"].dtype.kind in "biufc":
            result[f"{prop}_diff"] = np.abs(result[f"{prop}_left"] - result[f"{prop}_right"])
        else:
            result[f"{prop}_diff"] = (result[f"{prop}_left"] != result[f"{prop}_right"]).astype(int)
            result = pd.concat(
                [
                    result.drop(columns=[f"{prop}_left", f"{prop}_right"]), 
                    pd.get_dummies(result[[f"{prop}_left", f"{prop}_right"]], columns=[f"{prop}_left", f"{prop}_right"], dtype=int, drop_first=True)
                ],
                axis=1)
    return result.set_index("index")

In [ ]:
(1,2) in graph_block.edges()

In [ ]:
df = create_connection_data(graph_block)

In [ ]:
df

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from scipy import stats

In [ ]:
X = df.drop(columns=["connected"])
y = df["connected"]
model = LogisticRegression(random_state=0).fit(X, y)

In [ ]:
params = np.concatenate([model.intercept_, model.coef_.ravel()])

# Predicted probabilities
p = model.predict_proba(X)[:,1]

# Construct design matrix including intercept
X_design = np.column_stack([np.ones(X.shape[0]), X])

# Weight matrix diagonal
W = np.diag(p * (1 - p))

# Fisher Information matrix
Fisher = X_design.T @ W @ X_design

# Covariance matrix = inverse(Fisher)
cov_matrix = np.linalg.inv(Fisher)

# Standard errors = sqrt of diagonal
standard_errors = np.sqrt(np.diag(cov_matrix))

# Calculate z-scores and p-values
z_scores = params / standard_errors
p_values = 2 * (1 - stats.norm.cdf(abs(z_scores)))

# Create results DataFrame
features = X.columns
results = pd.DataFrame({
    'Feature': ['Intercept'] + list(features),
    'Coefficient': params,
    'Std Error': standard_errors,
    'z-value': z_scores,
    'p-value': p_values
})
results

In [ ]:
data = pd.DataFrame([
    {
        'type': 'Y - Y',
        'color_diff': 0,
        'color_left_green': 0,
        'color_left_yellow': 1,
        'color_right_green': 0,
        'color_right_yellow': 1
    },
    {
        'type': 'G - G',
        'color_diff': 0,
        'color_left_green': 1,
        'color_left_yellow': 0,
        'color_right_green': 1,
        'color_right_yellow': 0
    },
    {
        'type': 'B - B',
        'color_diff': 0,
        'color_left_green': 0,
        'color_left_yellow': 0,
        'color_right_green': 0,
        'color_right_yellow': 0
    },
    {
        'type': 'Y - G',
        'color_diff': 1,
        'color_left_green': 0,
        'color_left_yellow': 1,
        'color_right_green': 1,
        'color_right_yellow': 0
    },
    {
        'type': 'Y - B',
        'color_diff': 1,
        'color_left_green': 0,
        'color_left_yellow': 1,
        'color_right_green': 0,
        'color_right_yellow': 0
    },
    {
        'type': 'G - Y',
        'color_diff': 1,
        'color_left_green': 1,
        'color_left_yellow': 0,
        'color_right_green': 0,
        'color_right_yellow': 1
    },
    {
        'type': 'G - B',
        'color_diff': 1,
        'color_left_green': 1,
        'color_left_yellow': 0,
        'color_right_green': 0,
        'color_right_yellow': 0
    },
    {
        'type': 'B - G',
        'color_diff': 1,
        'color_left_green': 0,
        'color_left_yellow': 0,
        'color_right_green': 1,
        'color_right_yellow': 0
    },
    {
        'type': 'B - Y',
        'color_diff': 1,
        'color_left_green': 0,
        'color_left_yellow': 0,
        'color_right_green': 0,
        'color_right_yellow': 1
    }
])
data['probability'] = model.predict_proba(data.drop(columns=['type']))[:,1]
data


In [ ]:
pd.DataFrame([
    {'type': 'B - B', 'probability': 5/6},
    {'type': 'Y - Y', 'probability': 3/3},
    {'type': 'G - G', 'probability': 5/6},
    {'type': 'B - Y', 'probability': 2/12},
    {'type': 'B - G', 'probability': 2/16},
    {'type': 'Y - G', 'probability': 1/12}
])